In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import shutil
import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from glob import glob
import random
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
        
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.preprocessing.image as Image

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
# df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
fast_df = False
IMG_SIZE = 600

In [ ]:
study = pd.read_csv("/kaggle/input/siim-covid19-detection/train_study_level.csv")
study['id'] = study['id'].str.split('_',expand=True)[0]


In [ ]:
study

In [ ]:
print(study["Negative for Pneumonia"].sum())
print(study["Typical Appearance"].sum())
print(study["Indeterminate Appearance"].sum())
print(study["Atypical Appearance"].sum())

In [ ]:
#absolute difference
print(study["Typical Appearance"].sum()-study["Negative for Pneumonia"].sum())
print(study["Typical Appearance"].sum()-study["Indeterminate Appearance"].sum())
print(study["Typical Appearance"].sum()-study["Atypical Appearance"].sum())

In [ ]:
os.mkdir('/kaggle/working/real')
os.mkdir('/kaggle/working/augmented')
os.mkdir('/kaggle/working/real/train')
os.mkdir('/kaggle/working/augmented/train')
os.mkdir('/kaggle/working/real/train/Negative')
os.mkdir('/kaggle/working/real/train/Typical')
os.mkdir('/kaggle/working/real/train/Indeterminate')
os.mkdir('/kaggle/working/real/train/Atypical')
os.mkdir('/kaggle/working/augmented/train/Negative')
os.mkdir('/kaggle/working/augmented/train/Typical')
os.mkdir('/kaggle/working/augmented/train/Indeterminate')
os.mkdir('/kaggle/working/augmented/train/Atypical')


In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
save_dir = f'/kaggle/working/real/train/'

for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/train')):
    for file in filenames:
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size=600)
        entry = study.loc[study['id']==dirname.split("/")[-2]]
        if entry["Negative for Pneumonia"].item()==1:  
            im.save(os.path.join(save_dir+"Negative",entry['id'].item()+"_study.jpeg"))
        elif entry["Typical Appearance"].item()==1:  
            im.save(os.path.join(save_dir+"Typical",entry['id'].item()+"_study.jpeg"))
        elif entry["Indeterminate Appearance"].item()==1:  
            im.save(os.path.join(save_dir+"Indeterminate",entry['id'].item()+"_study.jpeg"))
        elif entry["Atypical Appearance"].item()==1:  
            im.save(os.path.join(save_dir+"Atypical",entry['id'].item()+"_study.jpeg"))



In [ ]:
l = len(os.listdir("/kaggle/working/real/train/Typical"))

In [ ]:
imgs_paths = sorted([os.path.abspath(os.path.join('/kaggle/working/real/train/Indeterminate/', p)) for p in os.listdir('/kaggle/working/real/train/Indeterminate/')])

li = l- len(imgs_paths)-200
print(len(imgs_paths))
print(li)
i = 0
final_path = "/kaggle/working/augmented/train/Indeterminate/"
while i<li:
    img_path = random.choice(imgs_paths)
    number = random.randint(1,3)
    image = tf.keras.preprocessing.image.load_img(img_path)
    if number == 1:
        image = tf.keras.preprocessing.image.random_rotation(Image.img_to_array(image),rg=30)
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_rot_"+str(i)+".jpeg",image,file_format="jpeg")
            
    if number == 2:
        image = tf.image.flip_left_right(Image.img_to_array(image))
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_flip_"+str(i)+".jpeg",image,file_format="jpeg")
                
    if number ==3:
        image= tf.keras.preprocessing.image.random_shift(Image.img_to_array(image),0.2,0.1)
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_trans_"+str(i)+".jpeg",image,file_format="jpeg")
    i += 1
              

In [ ]:
imgs_paths = sorted([os.path.abspath(os.path.join('/kaggle/working/real/train/Negative', p)) for p in os.listdir('/kaggle/working/real/train/Negative')])

li = l- len(imgs_paths)-50
print(len(imgs_paths))
print(li)
i = 0
final_path = "/kaggle/working/augmented/train/Negative/"
while i<li:
    img_path = random.choice(imgs_paths)
    number = random.randint(1,3)
    image = tf.keras.preprocessing.image.load_img(img_path)
    if number == 1:
        image = tf.keras.preprocessing.image.random_rotation(Image.img_to_array(image),rg=30)
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_rot_"+str(i)+".jpeg",image,file_format="jpeg")
            
    if number == 2:
        image = tf.image.flip_left_right(Image.img_to_array(image))
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_flip_"+str(i)+".jpeg",image,file_format="jpeg")
                
    if number ==3:
        image= tf.keras.preprocessing.image.random_shift(Image.img_to_array(image),0.2,0.1)
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_trans_"+str(i)+".jpeg",image,file_format="jpeg")
    i += 1
              

In [ ]:
imgs_paths = sorted([os.path.abspath(os.path.join('/kaggle/working/real/train/Atypical', p)) for p in os.listdir('/kaggle/working/real/train/Atypical')])

li = l- len(imgs_paths)-500
print(len(imgs_paths))
print(li)
i = 0
final_path = "/kaggle/working/augmented/train/Atypical/"
while i<li:
    img_path = random.choice(imgs_paths)
    number = random.randint(1,3)
    image = tf.keras.preprocessing.image.load_img(img_path)
    if number == 1:
        image = tf.keras.preprocessing.image.random_rotation(Image.img_to_array(image),rg=30)
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_rot_"+str(i)+".jpeg",image,file_format="jpeg")
            
    if number == 2:
        image = tf.image.flip_left_right(Image.img_to_array(image))
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_flip_"+str(i)+".jpeg",image,file_format="jpeg")
                
    if number ==3:
        image= tf.keras.preprocessing.image.random_shift(Image.img_to_array(image),0.2,0.1)
        tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+"_trans_"+str(i)+".jpeg",image,file_format="jpeg")
    i += 1
              

In [ ]:
imgs_paths = sorted([os.path.abspath(os.path.join('/kaggle/working/real/train/Atypical', p)) for p in os.listdir('/kaggle/working/real/train/Atypical')])


final_path = "/kaggle/working/augmented/train/Atypical/"
for img_path in imgs_paths:
    image = tf.keras.preprocessing.image.load_img(img_path)
    tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+".jpeg",image,file_format="jpeg")
    i += 1
              

In [ ]:
imgs_paths = sorted([os.path.abspath(os.path.join('/kaggle/working/real/train/Typical', p)) for p in os.listdir('/kaggle/working/real/train/Typical')])


final_path = "/kaggle/working/augmented/train/Typical/"
for img_path in imgs_paths:
    image = tf.keras.preprocessing.image.load_img(img_path)
    tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+".jpeg",image,file_format="jpeg")
    i += 1
              

In [ ]:
imgs_paths = sorted([os.path.abspath(os.path.join('/kaggle/working/real/train/Indeterminate', p)) for p in os.listdir('/kaggle/working/real/train/Indeterminate')])


final_path = "/kaggle/working/augmented/train/Indeterminate/"
for img_path in imgs_paths:
    image = tf.keras.preprocessing.image.load_img(img_path)
    tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+".jpeg",image,file_format="jpeg")
    i += 1
              

In [ ]:
imgs_paths = sorted([os.path.abspath(os.path.join('/kaggle/working/real/train/Negative', p)) for p in os.listdir('/kaggle/working/real/train/Negative')])


final_path = "/kaggle/working/augmented/train/Negative/"
for img_path in imgs_paths:
    image = tf.keras.preprocessing.image.load_img(img_path)
    tf.keras.preprocessing.image.save_img(final_path + img_path[:-4].split('/')[-1]+".jpeg",image,file_format="jpeg")
    i += 1
              

In [ ]:
real = 0
for i in os.listdir("/kaggle/working/real/train/"):
    print(i,len(os.listdir("/kaggle/working/real/train/"+i)))
    real += len(os.listdir("/kaggle/working/real/train/"+i))
print("-"*20)
print("total = ",real)
print()
print()
print()

augmented = 0
for i in os.listdir("/kaggle/working/augmented/train/"):
    print(i,len(os.listdir("/kaggle/working/augmented/train/"+i)))
    augmented += len(os.listdir("/kaggle/working/augmented/train/"+i))
print("-"*20)
print("total = ",augmented)
print()
print()
print()